1.Add  column to say yes or no whether the country is Mexico
2.Group by your new column and sum bytes used
3.Group by Country and use the sqlfunc.countDistinct functions to calculate the number of IP addresses seen in each country.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
df=spark.read.csv('challenge.csv',header=True,)

In [15]:
from pyspark.sql.functions import *
import pyspark.sql.functions as sqlfunc

In [3]:
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [13]:
df1=df.withColumn('IsMexico', when(df.Country=='Mexico','Yes').otherwise('No'))

In [20]:
df1.show()

+---------------+--------------+-----------------+----------+--------+
|     ip_address|       Country|      Domain Name|Bytes_used|IsMexico|
+---------------+--------------+-----------------+----------+--------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|      No|
| 119.239.207.13|         China|         youtu.be|        51|      No|
|  68.69.217.210|         China|        adobe.com|        10|      No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|      No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|      No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|      No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|      No|
| 127.242.24.138|         China| surveymonkey.com|       123|      No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|      No|
|   237.54.11.63|         China|       amazon.com|        83|      No|
| 252.141.157.25|         Japan|      cornell.edu|       374|      No|
|185.2

In [23]:
df2=df1.groupBy('IsMexico').agg(sqlfunc.sum(df1.Bytes_used).alias('total_used'))

In [24]:
df2.show()

+--------+----------+
|IsMexico|total_used|
+--------+----------+
|      No|  508076.0|
|     Yes|    6293.0|
+--------+----------+



In [25]:
df3=df.groupBy('Country').agg(sqlfunc.countDistinct(df.ip_address).alias('number_of_ips'))

In [26]:
df3.sort(col('number_of_ips').desc()).show()

+--------------+-------------+
|       Country|number_of_ips|
+--------------+-------------+
|         China|          172|
|     Indonesia|          114|
|   Philippines|           65|
|        Russia|           56|
|        Brazil|           35|
|        Poland|           31|
|        Sweden|           28|
|         Japan|           25|
|      Portugal|           23|
|Czech Republic|           23|
|        France|           21|
|          Peru|           19|
|      Colombia|           17|
| United States|           15|
|       Ukraine|           14|
|     Argentina|           14|
|        Mexico|           13|
|      Thailand|           12|
|       Nigeria|           11|
|        Canada|           11|
+--------------+-------------+
only showing top 20 rows

